## Entrenamiento del modelo

### Importación de librerías

Datos y gráficos

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pickle
import yaml
import os

Preprocesado y modelado

In [4]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

### Warnings y display de la dataframe

In [6]:
pd.set_option("max_colwidth", 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

import warnings
warnings.filterwarnings("ignore")

### Importación de dataframes

In [7]:
directorio_actual = os.getcwd()

In [8]:

df_num_NS = pd.read_csv(os.path.join(directorio_actual, '..', 'data','processed', 'df_num_NS.csv'))
df_NS = pd.read_csv(os.path.join(directorio_actual, '..', 'data','processed', 'df_NS.csv'))

X = df_num_NS.drop(['price'],axis=1)
y = df_num_NS['price']


### Entrenamiento y evaluación

Escalado de X

In [9]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Entrenamiento

In [10]:
#El numero de cluster, será 50 que es la cifra aproximada que necesita para predecir el 15% de las viviendas sin error
kmeans = KMeans(n_clusters=50) 
clusters = kmeans.fit_predict(X_scaled)

In [11]:
#Creo una columna los clusters asignados a cada registro
df_num_NS['cluster'] = clusters

In [12]:
#Realizo un train split del 15 por ciento
df_train_NS, df_test_NS = train_test_split(df_num_NS,test_size=0.15)

In [13]:
#grupoby por la columna cluster y le agrego la media del precio por cada cluster
precio_medio_por_cluster = df_train_NS.groupby(['cluster'],as_index=False)['price'].agg('mean')

In [14]:
#creo un diccionario con cada cluster y su media de precio
mapeo_cluster = dict(zip(precio_medio_por_cluster['cluster'], precio_medio_por_cluster['price']))

In [16]:
#creo la columna predicción, donde segun el cluster de cada registro, se le adjudica su media de precio
df_num_NS['prediccion'] = df_num_NS['cluster'].map(mapeo_cluster)

In [17]:
print('MAE',mean_absolute_error(df_num_NS['price'],df_num_NS['prediccion']))
print('MAPE:',mean_absolute_percentage_error(df_num_NS['price'],df_num_NS['prediccion']))
print('RMSE:', np.sqrt(mean_squared_error(df_num_NS['price'],df_num_NS['prediccion'])))

MAE 210.07014335864608
MAPE: 0.18154241109243072
RMSE: 324.36636963222253


### Predicciones

In [18]:
#función similar a las anteriores
def calcular_alquiler_NS(tamaño,poblacion_distrito,parking,tipo_vivienda,piscina,habitaciones,baños):
    
    dicc_tipo = {'Estudio' : 1,'Piso' : 2,'Adosado' : 3,'Duplex' : 4,'Ático' : 5,'Casa pareada' : 6,'Chalet independiente' : 7}
    dicc = {}
    
    dicc['size'] = tamaño

    for i, pob_dis in enumerate(df_NS['poblacion / distrito']):
        if pob_dis == poblacion_distrito:
            for n, codigo_dist in enumerate(df_NS['precio_area / distrito']):
                if n == i:
                    dicc['precio_area / distrito'] = codigo_dist
                    break
    if parking == 'No':            
        dicc['parking'] = 0
    else:
        dicc['parking'] = 1

    for tipo,codigo in dicc_tipo.items():
        if tipo == tipo_vivienda:
            dicc['codigo_tipo'] = codigo

    if piscina == 'No':            
        dicc['piscina'] = 0
    else:
        dicc['piscina'] = 1

    
    dicc['total_rooms'] = habitaciones + baños
    
    
    return(dicc)    
    

In [19]:
#creo una nueva predicción y la comvierto en dataframe
nuevo_input = calcular_alquiler_NS(100,'Sevilla / Nervión','Si','Casa pareada','No',2,1)
nuevo_input = pd.DataFrame([nuevo_input])

In [20]:
#concateno la X, con la nueva predicción y reseteo indices creando la variable, nueva_df
nueva_df = pd.concat([X,nuevo_input])
nueva_df.reset_index(inplace=True)
nueva_df.drop(['index'],axis=1,inplace=True)

In [21]:
#comenzamos con 250 cluster que es la cantidad, que hemos observado que da un erro muy bajo y no tarda mucho en la busqueda
n = 250
scaler = StandardScaler()
nueva_df_scaled = scaler.fit_transform(nueva_df)
kmeans = KMeans(n_clusters = n) 
clusters = kmeans.fit_predict(nueva_df_scaled)
nueva_df['cluster'] = clusters

'''mediante un bucle while, buscaremos el número de cluster necesario,
para que el nuevo imput, no quede solo en un cluster y no obtenga precio,
ya que al estar solo, no habria registros a los que sacarle la media'''
while len(nueva_df[nueva_df['cluster'] == nueva_df['cluster'][len(nueva_df)-1]]) == 1:
    n = n - 1
    scaler = StandardScaler()
    nueva_df_scaled = scaler.fit_transform(nueva_df)
    kmeans = KMeans(n_clusters = n) 
    clusters = kmeans.fit_predict(nueva_df_scaled)
    nueva_df['cluster'] = clusters
    
print(nueva_df['cluster'].nunique())
print(len(nueva_df[nueva_df['cluster'] == nueva_df['cluster'][len(nueva_df)-1]]))

220
4


In [22]:
#dividimos la nueva dataframe, sacando así, el ultimo registro, que es nuestro nuevo imput
nueva_df_train, nueva_df_test = train_test_split(nueva_df,test_size=1, shuffle=False)
nueva_df_train = pd.concat([nueva_df_train,df_num_NS['price']],axis=1)

In [23]:
#al igual que en la evaluación, agrupamos por cluster, creamos diccionario y añadimos la columna predicciones
precio_medio_por_cluster = nueva_df_train.groupby(['cluster'],as_index=False)['price'].agg('mean')
mapeo_cluster = dict(zip(precio_medio_por_cluster['cluster'], precio_medio_por_cluster['price']))
nueva_df_train['prediccion'] = nueva_df_train['cluster'].map(mapeo_cluster)

In [24]:
'''recorremos la columna cluster de la dataframe anterior 
y cuando el cluster coincida con el del nuevo imput,
le asignamos al nuevo input el precio medio de su cluster'''
for i,cluster in enumerate(nueva_df_train['cluster']):
    if cluster == nueva_df_test['cluster'][len(nueva_df)-1]:
        prediccion = nueva_df_train['prediccion'][i]
        break 

In [25]:
prediccion

2000.0